In [156]:
import spacy
import pdfminer
import re
import os
import pandas as pd

In [157]:
import pdf2txt

In [158]:
def convert_pdf(f):
    output_filename = os.path.basename(os.path.splitext(f)[0]) + '.txt'
    output_filepath = os.path.join("output/txt/", output_filename)
    pdf2txt.main(args=[f, "--outfile", output_filepath])
    print(output_filepath + " saved successfully!!!")
    return open(output_filepath).read()

In [159]:
nlp = spacy.load("en_core_web_sm")

In [160]:
#phone_num credit https://stackoverflow.com/a/3868861

In [161]:
result_dict = {'name': [], 'phone': [], 'email': [], 'skill': []}
names = []
phones = []
emails = []
skills = []

In [162]:
def parse_content(text):
    skillset = re.compile("python|java|sql|hadoop|tableau")
    phone_num = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")
    doc = nlp(text)
    name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]
    parts = name.split(" ")
    name = parts[0] + ' ' + parts[1]
    print('['+name+']')
    email = [word for word in doc if word.like_email == True][0]
    print(email)
    phone = str(re.findall(phone_num, text.lower()))
    skills_list = re.findall(skillset, text.lower())
    unique_skills_list = str(set(skills_list))
    names.append(name)
    emails.append(email)
    phones.append(phone)
    skills.append(unique_skills_list)
    print("Extraction completed successfully!!!")

<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
C:\Users\krist\AppData\Local\Temp/ipykernel_11944/2413439028.py:5: SyntaxWarning: "is" with a literal. Did you mean "=="?
  name = [entity.text for entity in doc.ents if entity.label_ is "PERSON"][0]


In [163]:
for file in os.listdir('resumes/'):
    if file.endswith('.pdf'):
        print('Reading...' + file)
        txt = convert_pdf(os.path.join('resumes/', file))
        parse_content(txt)

Reading...Alisson ParkerCV.pdf
output/txt/Alisson ParkerCV.txt saved successfully!!!
[Alisson Parker-Wright]
alli1414parks@mail.com
Extraction completed successfully!!!
Reading...AshleyMilesCV.pdf
output/txt/AshleyMilesCV.txt saved successfully!!!
[Ashley Miles]
ashleymiles@memail.com
Extraction completed successfully!!!
Reading...John DominicCV.pdf
output/txt/John DominicCV.txt saved successfully!!!
[John Dominic]
johndominic@mail.com
Extraction completed successfully!!!


In [164]:
result_dict['name'] = names
result_dict['phone'] = phones
result_dict['email'] = emails
result_dict['skill'] = skills
#print(result_dict)

In [165]:
result_df = pd.DataFrame(result_dict)
result_df

,name,phone,email,skill
0,Alisson Parker-Wright,['8569878511'],alli1414parks@mail.com,"{'python', 'java', 'tableau'}"
1,Ashley Miles,['6592251422'],ashleymiles@memail.com,"{'sql', 'tableau'}"
2,John Dominic,['7877756411'],johndominic@mail.com,"{'python', 'java', 'hadoop'}"


In [166]:
result_df.to_csv('output/csv/parsed_resumes.csv')